In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime

#Visualisation libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
import plotly.offline as py
!pip install pycountry
import pycountry
py.init_notebook_mode(connected=True)
import folium 
from folium import plugins

# Image
import numpy as np
from PIL import Image

# Animation
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML

# Graphics in retina format 
%config InlineBackend.figure_format = 'retina' 

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = 8, 5


# Disable warnings in Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

     |████████████████████████████████| 10.1 MB 25.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=7c32945827fbda6c10d0eafc5bd14900053ff07b0222f7c41b2a3e2d8bf7b79d
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry


In [ ]:
df = pd.read_csv('Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')
df.head()

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0


In [ ]:
# Creando una copia del primer dataframe-df
df1 = df.copy()
df1.drop('S.NO',axis=1,inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  301 non-null    object 
 1   Nationality           301 non-null    object 
 2   Current Rank          301 non-null    int64  
 3   Previous Year Rank    277 non-null    object 
 4   Sport                 301 non-null    object 
 5   Year                  301 non-null    int64  
 6   earnings ($ million)  301 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 16.6+ KB


In [ ]:
# Conviritiendo el texto a datetime64
df1['Year'] = df1['Year'].apply(pd.to_datetime,format='%Y')

#Set Date column as the index column.
df1['year'] = pd.DatetimeIndex(df1['Year']).year  
df1.set_index('year', inplace=True)
df1.drop('Year',axis=1,inplace=True)

# Convirtiendo el texto de la sport column a mayusculas
df1['Sport'] = df1['Sport'].str.upper()
df1

# df is the original dataframe while df1 is a copy where the Date has been set as an index column

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
1990,Mike Tyson,USA,1,NaN,BOXING,28.6
1990,Buster Douglas,USA,2,NaN,BOXING,26.0
1990,Sugar Ray Leonard,USA,3,NaN,BOXING,13.0
1990,Ayrton Senna,Brazil,4,NaN,AUTO RACING,10.0
1990,Alain Prost,France,5,NaN,AUTO RACING,9.0
...,...,...,...,...,...,...
2020,Stephen Curry,USA,6,9,BASKETBALL,74.4
2020,Kevin Durant,USA,7,10,BASKETBALL,63.9
2020,Tiger Woods,USA,8,11,GOLF,62.3


In [ ]:
#Cambiando nombre de Países mal nombrados:
##Como Irlanda del Norte es diferente a Irlanda, se considerarán como países distintos.
##Como filipino hace referncia a que es de Filipinas, se remplaza el termino filipino por filipinas.
dfcopy = df1.copy()
change = dfcopy['Nationality'].map(lambda x: 'Philippines' if x =='Filipino' else x)

dfcopy.drop(columns='Nationality', inplace=True)
dfc = pd.concat([dfcopy, change], axis=1)
dfc = dfc[['Name', 'Nationality', 'Current Rank', 'Previous Year Rank', 'Sport', 'earnings ($ million)']]
dfc.head()

#####   dfc es la df que se usa para los siguientes ejercicios    #####




,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
1990,Mike Tyson,USA,1,NaN,BOXING,28.6
1990,Buster Douglas,USA,2,NaN,BOXING,26.0
1990,Sugar Ray Leonard,USA,3,NaN,BOXING,13.0
1990,Ayrton Senna,Brazil,4,NaN,AUTO RACING,10.0
1990,Alain Prost,France,5,NaN,AUTO RACING,9.0


In [ ]:
#Limpieza de doble deporte en un atleta:
dfsep = dfc.copy()

sepExpanded = dfsep['Sport'].str.split(' / ', expand=True)
sepExpanded.columns = ['Sport', '2°Sport']
dfsep.drop(columns='Sport', inplace=True)
dfs = pd.concat([dfsep, sepExpanded], axis=1)
dfs = dfs[['Name', 'Nationality', 'Current Rank', 'Previous Year Rank', 'Sport', '2°Sport', 'earnings ($ million)']]
dfs


,Name,Nationality,Current Rank,Previous Year Rank,Sport,2°Sport,earnings ($ million)
year,,,,,,,
1990,Mike Tyson,USA,1,NaN,BOXING,None,28.6
1990,Buster Douglas,USA,2,NaN,BOXING,None,26.0
1990,Sugar Ray Leonard,USA,3,NaN,BOXING,None,13.0
1990,Ayrton Senna,Brazil,4,NaN,AUTO RACING,None,10.0
1990,Alain Prost,France,5,NaN,AUTO RACING,None,9.0
...,...,...,...,...,...,...,...
2020,Stephen Curry,USA,6,9,BASKETBALL,None,74.4
2020,Kevin Durant,USA,7,10,BASKETBALL,None,63.9
2020,Tiger Woods,USA,8,11,GOLF,None,62.3


In [169]:
#Limpieza de los deportes repetidos
##Nascar es un tipo de disciplina del AUTO RACING (automovilismo en español), por lo que el termino NASCAR se cambia a AUTO RACING (NASCAR)
##NBA hace referencia al basketball, por lo que NBA se cambia a basketball
##NFL hace referencia al American Football, por lo que NFL se cambia a American Football
##F1 RACING y F1 MOTORSPORTS son F1, y como F1 es una disciplina del AUTO RACING, se renombran como AUTO RACING (F1)
##ICE HOCKEY es igual a HOCKEY

dfs['Sport'] = dfs['Sport'].str.replace('NBA', 'BASKETBALL')
dfs['Sport'] = dfs['Sport'].str.replace('NFL', 'AMERICAN FOOTBALL')

daa = dfs.copy()

ch1 = daa['Sport'].map(lambda x: 'ssf' if x =='AUTO RACING (NASCAR)' else x)

daa.drop(columns='Sport', inplace=True)
dfs1 = pd.concat([daa, ch1], axis=1)
dfs1 = dfs1[['Name', 'Nationality', 'Current Rank', 'Previous Year Rank', 'Sport', 'earnings ($ million)']]

dfs1['Sport'] = dfs1['Sport'].str.replace('NASCAR', 'AUTO RACING (NASCAR)')
dfs1['Sport'] = dfs1['Sport'].str.replace('ssf', 'AUTO RACING (NASCAR)')
dfs1['Sport'] = dfs1['Sport'].str.replace('ICE HOCKEY', 'HOCKEY')
dfs1['Sport'] = dfs1['Sport'].str.replace('F1 RACING', 'AUTO RACING (F1)')
dfs1['Sport'] = dfs1['Sport'].str.replace('F1 MOTORSPORTS', 'AUTO RACING (F1)')
dfs1




,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
1990,Mike Tyson,USA,1,NaN,BOXING,28.6
1990,Buster Douglas,USA,2,NaN,BOXING,26.0
1990,Sugar Ray Leonard,USA,3,NaN,BOXING,13.0
1990,Ayrton Senna,Brazil,4,NaN,AUTO RACING,10.0
1990,Alain Prost,France,5,NaN,AUTO RACING,9.0
...,...,...,...,...,...,...
2020,Stephen Curry,USA,6,9,BASKETBALL,74.4
2020,Kevin Durant,USA,7,10,BASKETBALL,63.9
2020,Tiger Woods,USA,8,11,GOLF,62.3


In [191]:
dfbob = dfs1.copy()
dfbob.to_csv('file.csv', index= False)

In [187]:
#Limpieza de los valores en Previous Year Rank
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('>10', '11')
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('>20', '21')
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('>14', '15')
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('?', '0')
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('>30', '31')
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('>40', '41')
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('>100', '101')
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('not ranked', '0')
dfs1['Previous Year Rank'] = dfs1['Previous Year Rank'].str.replace('none', '0')

# **Deportistas cuyo ranking ha subido al menos dos lugares entre 2010 y 2020**

In [188]:
df11 = dfs1.copy()

data_1020 = df11[df11.index > 2009 ]
data_1020 = data_1020[data_1020.index < 2021 ]

#data_1020['Previous Year Rank'] = data_1020['Previous Year Rank'].str.replace('nan', '0')
data_1020

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
2010,Tiger Woods,USA,1,1,GOLF,105.0
2010,Floyd Mayweather,USA,2,11,BOXING,65.0
2010,Kobe Bryant,USA,3,2,BASKETBALL,48.0
2010,Phil Mickelson,USA,4,6,GOLF,46.0
2010,David Beckham,UK,5,5,SOCCER,43.7
...,...,...,...,...,...,...
2020,Stephen Curry,USA,6,9,BASKETBALL,74.4
2020,Kevin Durant,USA,7,10,BASKETBALL,63.9
2020,Tiger Woods,USA,8,11,GOLF,62.3


# **Atleta con el menor número de apariciones y mayores ganancias**

In [180]:
df44 = dfs1.copy()
#Ganancia máxima de cada atleta:
at_max = pd.pivot_table(df44, index='Name', values="earnings ($ million)", aggfunc='max')

at_max['Ganancia máxima ($ million)'] = pd.Float64Index(at_max['earnings ($ million)'])
at_max.drop('earnings ($ million)',axis=1,inplace=True)
at_max = at_max.sort_values(by='Ganancia máxima ($ million)',ascending=False)[:]
at_max


,Ganancia máxima ($ million)
Name,
Floyd Mayweather,300.0
Manny Pacquiao,160.0
Lionel Messi,127.0
Tiger Woods,115.0
Cristiano Ronaldo,109.0
...,...
Michael Moorer,12.1
Joe Montana,11.5
"Donovan ""Razor"" Ruddock",10.2


In [182]:
df44 = dfs1.copy()
at_min = df44['Name'].value_counts().to_frame()[:]

at_min.info()
at_min.style.background_gradient(cmap='Reds')

##Respuesta=Canelo Alvarez

<class 'pandas.core.frame.DataFrame'>
Index: 82 entries, Tiger Woods to Carson Wentz
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Name    82 non-null     int64
dtypes: int64(1)
memory usage: 1.3+ KB


,Name
Tiger Woods,19
Michael Jordan,19
Kobe Bryant,14
LeBron James,13
Michael Schumacher,13
Roger Federer,12
Phil Mickelson,11
Cristiano Ronaldo,10
Shaquille O'Neal,9
Lionel Messi,9


# **País con mayor número de deportes con atletas en el dataset**

In [ ]:
df55 = dfs1.copy()
df55 = df55[['Nationality', 'Sport']]
paises = pd.pivot_table(df55, index=['Nationality', 'Sport'])
paises = paises.reset_index('Nationality')
paises = paises['Nationality'].value_counts().to_frame()[:6]

paises = paises.rename(columns={'Nationality': 'Número de deportes'})
paises.style.background_gradient(cmap='Reds')

### Quizás las otras disciplinas podrían ser convertidas a AUTO RACING y eso eliminaría 1 deporte
### pero para no evitar las especificaciones de disciplinas, se deja AUTO RACING como diferentes a
### AUTO RACING (NASCAR) y AUTO RACING (F1) 



,Número de deportes
USA,9
Canada,3
UK,2
Brazil,2
Germany,2
Northern Ireland,1


# **¿Cuántos atletas por deporte tiene cada país?**

In [ ]:
df77 = dfs1.copy()
df77 = df77[['Nationality', 'Sport', 'Name' ]]
atletas = pd.pivot_table(df77, index=['Nationality', 'Sport', 'Name' ])
atletas = atletas.reset_index('Nationality')
atletas = atletas['Nationality'].value_counts().to_frame()[:]

atletas = atletas.rename(columns={'Nationality': 'Número de atletas por deporte'})
atletas.style.background_gradient(cmap='Reds')

,Número de atletas por deporte
USA,58
Canada,4
UK,3
Brazil,3
Germany,2
Northern Ireland,1
Switzerland,1
Spain,1
Serbia,1
Russia,1


# **Ganancia mínima y máxima (dentro del dataset) por deporte y por país**

In [ ]:
#Consideraremos al Datasat como df porque es el archivo original.
df['Sport'] = df['Sport'].str.upper()

#Ganancia mínima y máxima por deporte
sport_min = pd.pivot_table(df, index='Sport', values='earnings ($ million)', aggfunc='min')
sport_min['Ganancia mínima ($ million)'] = pd.Float64Index(sport_min['earnings ($ million)'])
sport_min.drop('earnings ($ million)',axis=1,inplace=True)

sport_max = pd.pivot_table(df, index='Sport', values="earnings ($ million)", aggfunc='max')
sport_max['Ganancia máxima ($ million)'] = pd.Float64Index(sport_max['earnings ($ million)'])
sport_max.drop('earnings ($ million)',axis=1,inplace=True)

sport_maxmin = pd.merge(sport_max, sport_min, how="outer", on="Sport")
sport_maxmin = sport_maxmin.sort_values(by='Ganancia máxima ($ million)',ascending=False)[:]
sport_maxmin

,Ganancia máxima ($ million),Ganancia mínima ($ million)
Sport,,
BOXING,300.0,8.1
SOCCER,127.0,28.0
GOLF,115.0,8.1
TENNIS,106.3,8.5
MMA,99.0,99.0
AMERICAN FOOTBALL,89.5,29.0
BASKETBALL,89.0,8.1
F1 RACING,80.0,34.0
F1 MOTORSPORTS,75.0,33.0


In [ ]:
#Consideraremos al Datasat como df porque es el archivo original.
df['Nationality'] = df['Nationality'].str.upper()

#Ganancia mínima y máxima por país
nation_min = pd.pivot_table(df, index='Nationality', values="earnings ($ million)", aggfunc='min')
nation_min['Ganancia mínima ($ million)'] = pd.Float64Index(nation_min['earnings ($ million)'])
nation_min.drop('earnings ($ million)',axis=1,inplace=True)

nation_max = pd.pivot_table(df, index='Nationality', values="earnings ($ million)", aggfunc='max')
nation_max['Ganancia máxima ($ million)'] = pd.Float64Index(nation_max['earnings ($ million)'])
nation_max.drop('earnings ($ million)',axis=1,inplace=True)

nation_maxmin = pd.merge(nation_max, nation_min, how="outer", on='Nationality')
nation_maxmin = nation_maxmin.sort_values(by='Ganancia máxima ($ million)',ascending=False)[:]
nation_maxmin


,Ganancia máxima ($ million),Ganancia mínima ($ million)
Nationality,,
USA,300.0,8.1
PHILIPPINES,160.0,40.0
ARGENTINA,127.0,32.3
PORTUGAL,109.0,38.0
SWITZERLAND,106.3,35.0
BRAZIL,105.0,10.0
IRELAND,99.0,99.0
MEXICO,94.0,94.0
GERMANY,80.0,15.0


# **Atleta con mayores ganancias por deporte por década**

In [178]:
#Primera decada 1990-2000
p_decada = dfs1[dfs1.index < 2001]
p_decadag = pd.pivot_table(p_decada, index='Sport', values="earnings ($ million)", aggfunc='max')

#Atletas con las mayores ganancias en la primera decada 1990-2000
ant = dfs1[['Name', 'Sport','earnings ($ million)']]
p_max = pd.merge(ant, p_decadag, how="inner", on=["Sport", "earnings ($ million)"])

p_max['Década'] = '1990-2000'
p_max.set_index('Década', inplace=True)


#Segunda decada 2000-2010
s_decada = dfs1[dfs1.index > 2000 ]
s_decada = s_decada[s_decada.index < 2011 ]

s_decadag = pd.pivot_table(s_decada, index='Sport', values="earnings ($ million)", aggfunc='max')

#Atletas con las mayores ganancias en la segunda década 2000-2010
bnt = dfs1[['Name', 'Sport','earnings ($ million)']]
s_max = pd.merge(bnt, s_decadag, how="inner", on=["Sport", "earnings ($ million)"])

s_max['Década'] = '2000-2010'
s_max.set_index('Década', inplace=True)

#Tercera decada 2010-2020
t_decada = dfs1[dfs1.index > 2010 ]
t_decada = t_decada[t_decada.index < 2021 ]

t_decadag = pd.pivot_table(t_decada, index='Sport', values="earnings ($ million)", aggfunc='max')

#Atletas con las mayores ganancias en la tercera década 2010-2020
tn = dfs1[['Name', 'Sport','earnings ($ million)']]
t_max = pd.merge(tn, t_decadag, how="inner", on=["Sport", "earnings ($ million)"])

t_max['Década'] = '2010-2020'
t_max.set_index('Década', inplace=True)

#Atleta con mayores ganancias por cada deporte por década
atletas_decada = pd.concat([p_max, s_max, t_max])

atletas_decada = pd.pivot_table(atletas_decada, index=['Década', 'Sport', 'Name'],values="earnings ($ million)")

atletas_decada.style.background_gradient(cmap='Reds') 

# **Ganancia total por cada deporte por cada año**

In [176]:
df2=dfs1.copy()

#Ganancia total por cada deporte por cada año:
ganacia_sport = pd.pivot_table(df2, index=['year', 'Sport'],values="earnings ($ million)", aggfunc='sum')
ganacia_sport = ganacia_sport.sort_values(by="year",ascending=False)[:]

ganacia_sport.style.background_gradient(cmap='Reds')  

